# Transformer training (chantier en cours)

Imports

In [9]:
%load_ext autoreload
%autoreload 2

from transformer_model import *
import nltk
import sys
sys.path.append("../../..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from statapp.common.preprocessing import load_all_data, encode_data, split_into_X_y

In [11]:
from statapp.common.sampling import sample_token_sequence

Preprocessing maison bien moche pour le moment... ^^ (essai avec un encodage sur les mots) Les données sont placées directement dans le dossier du notebook

In [14]:
text = load_all_data("data/fr.train.top1M.txt", sample=0.0000006)

tokens = nltk.word_tokenize(text)

In [15]:
len(tokens)

36

In [40]:
vocab = list(set(tokens))

vocab_size = len(vocab)

vocab_numbers = dict(zip(vocab, range(0,len(vocab))))
vocab_numeroted = dict(zip(range(0,len(vocab)), vocab))
tokens_numbers = np.array([vocab_numbers[tokens[i]] for i in range(len(tokens))])

tokens_numbers_sequences = np.array([ tokens_numbers[i:i+max_length] for i in range(len(tokens_numbers)-max_length+1)])
tokens_numbers_sequences = torch.tensor(tokens_numbers_sequences , dtype=torch.int64)

nb_sequences =  tokens_numbers_sequences.shape[0]

In [41]:
vocab_size

28

Apprentissage du modèle

In [42]:
LMtransformer = Transformer(vocab_size, Decoder(MultiHeadAttention(nb_heads, head_size, vector_size), FeedforwardNetwork(vector_size, ffn_hidden_size)))

#Correspond à utiliser l'entropie croisée puisque les sorties sont des log_softmax
#et l'entropie croisée = nll_loss(log_softmax(.), target)
criterion = nn.NLLLoss()
optimizer = optim.Adam(LMtransformer.parameters(), lr=0.01)

In [43]:
def train_model(nb_epochs, batch_size):
    
    #What is this ?? I don't remember. Make grad required ?
    LMtransformer.train()
    
    step = max(1,((len(tokens)-max_length)/batch_size)//5)
    
    for epoch in range(nb_epochs):
        
        running_loss = 0
        
        randperm = torch.randperm(nb_sequences)
        randperm = randperm[:(nb_sequences//batch_size)*batch_size]
        batchs_indices = randperm.reshape(nb_sequences//batch_size, batch_size)
        
        for i, batch_indices in enumerate(batchs_indices):
            
            batch = tokens_numbers_sequences[batch_indices]
            optimizer.zero_grad()
            output = LMtransformer(batch)
            loss = criterion(output[:,:-1].reshape(-1, vocab_size), batch[:,1:].flatten())

            loss.backward()
            optimizer.step()
            
            #Il faudrait adapter les affichages en fonction du nombre de batchs total
            running_loss += loss.item()
            if i % step == step-1:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / step))
                running_loss = 0.

Test d'overfitting sur un cas ultrasimplifié (5 tokens, longueur de séquence 1, 3 decoders, 2 heads) :
- En observant les sorties le modèle a bien appris et overfitte ! (loss à 0 au bout de 5-6 epochs) 

In [49]:
train_model(10,10)

[1,     1] loss: 0.063
[1,     2] loss: 0.069
[2,     1] loss: 0.026
[2,     2] loss: 0.054
[3,     1] loss: 0.021
[3,     2] loss: 0.088
[4,     1] loss: 0.050
[4,     2] loss: 0.060
[5,     1] loss: 0.018
[5,     2] loss: 0.015
[6,     1] loss: 0.067
[6,     2] loss: 0.010
[7,     1] loss: 0.052
[7,     2] loss: 0.007
[8,     1] loss: 0.034
[8,     2] loss: 0.059
[9,     1] loss: 0.045
[9,     2] loss: 0.036
[10,     1] loss: 0.023
[10,     2] loss: 0.097


Génération

In [21]:
#bidouilles pour adapter nos fonctions aux fonctions common codées par Nathra 
#(sequence list of ints en entree, list of probas en sortie)
#(Faire mieux plus tard)
def LMtransformerprediction(listints):
    return np.exp(LMtransformer(torch.tensor([listints]))[0][-1].tolist())

In [22]:
print(vocab)

['apres', 'age', 'dieu', 'd', '.', 'vrai', 'l', 'esprit', 'nom', 'par', 'il', 'de', 'persuade', 'barcelone', 'avoir', 'prophetique', 'ans', 'est', 'la', '31', 'alors', 'touche', 'du', 'a', 'obtenu', "'", 'connaissance', ',']


In [23]:
prev_seq = ['a','l','age','de']
prev_seq_numbers = [vocab_numbers[token] for token in prev_seq]
i=np.argmax(LMtransformerprediction(prev_seq_numbers))
tokens_pred = vocab_numeroted[i] 
print(tokens_pred)

31


In [24]:
def gen_seq(prev_seq):
    with torch.no_grad():
        prev_seq_numbers = [vocab_numbers[token] for token in prev_seq]
        sample_token_seq = sample_token_sequence(LMtransformerprediction, prev_seq_numbers)
        tokens_pred = [vocab_numeroted[i] for i in sample_token_seq]
        print(' '.join(tokens_pred)) 

In [25]:
def gen_seq_maison(prev_seq):
    with torch.no_grad():
        prev_seq_numbers = [vocab_numbers[token] for token in prev_seq]
        sample_token_seq = sample_token_sequence(LMtransformerprediction, prev_seq_numbers)
        tokens_pred = [vocab_numeroted[i] for i in sample_token_seq]
        print(' '.join(tokens_pred))

In [26]:
gen_seq(['il'])

100%|███████████████████████████████████████| 100/100 [00:00<00:00, 103.08it/s]


est touche par l ' esprit prophetique apres avoir obtenu la connaissance du vrai nom de dieu . il est alors dieu . il est alors persuade d ' esprit prophetique apres avoir obtenu la connaissance 31 ans , a barcelone , a barcelone , il est touche par l ' esprit prophetique apres avoir obtenu la connaissance du vrai nom de 31 ans , d ' esprit prophetique apres avoir . il est alors persuade d dieu . il est alors persuade il est alors persuade d ' esprit prophetique apres avoir obtenu la connaissance du vrai nom de


In [28]:
gen_seq(['a','l','age','de','31','ans'])

100%|███████████████████████████████████████| 100/100 [00:00<00:00, 113.68it/s]


, il est touche par l ' esprit prophetique apres avoir obtenu la connaissance du vrai nom de 31 ans , a barcelone , a l ' esprit prophetique apres avoir obtenu la connaissance du vrai nom de dieu . il est touche par l ' esprit prophetique apres avoir obtenu la connaissance du vrai nom de dieu . il est alors persuade prophetique apres avoir . il est alors persuade d par l ' esprit prophetique apres avoir obtenu la connaissance du vrai nom de dieu . il est alors dieu . il est alors 31 ans , a


In [29]:
gen_seq(['barcelone',',','il','est','touche','par','l'])

100%|███████████████████████████████████████| 100/100 [00:00<00:00, 108.84it/s]


' esprit prophetique apres avoir obtenu la connaissance du vrai nom de dieu . il est alors 31 ans , a barcelone , il est touche par l ' esprit prophetique apres avoir obtenu la connaissance du vrai nom de dieu . il est touche par l ' esprit prophetique apres avoir obtenu la connaissance du vrai nom de dieu . il est touche par l ' esprit prophetique apres avoir obtenu la connaissance du vrai nom de dieu . il est alors persuade d ' esprit prophetique apres avoir d dieu . d dieu . d dieu . il


In [27]:
print(' '.join(tokens))

a l ' age de 31 ans , a barcelone , il est touche par l ' esprit prophetique apres avoir obtenu la connaissance du vrai nom de dieu . il est alors persuade d '
